In [1]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import plotly.express as px
import re
from sklearn.preprocessing import MaxAbsScaler

In [49]:
def scaler_data(data):
    scaler = MaxAbsScaler()
    data_transform = scaler.fit_transform(data)
    df_transform = pd.DataFrame(data=data_transform, columns=["x", "y"])
    return df_transform["y"].values

In [50]:
def create_data(list_docs_to_read, name_folder):
    list_df = []

    raw_data_values_WT = []
    X_value_WT = []
    raw_data_values_MT = []
    X_value_MT = []

    meta_data = []
    
    for element in list_docs_to_read:

        df = pd.read_csv(f"{name_folder}{element}", sep="   ")
        df.columns = ["Wavenumber", "RRI"] 
        df["RRI-MAS"] = scaler_data(df)
        df["source"] = element
        row = df["RRI"].values.tolist()
        row.insert(0, element)

        x_values = df["Wavenumber"].values.tolist()
        x_values.insert(0, element)

        if "MD" in element:
            raw_data_values_MT.append(row)
            X_value_MT.append(x_values)
        else:
            raw_data_values_WT.append(row)
            X_value_WT.append(x_values)

        meta_data_row = {
            "source" : element,
            "is_mutant" : True if "MD" in element else False,
            "length" : df["Wavenumber"].shape[0],
            "min_x" : df["Wavenumber"].min(),
            "max_x" : df["Wavenumber"].max(),
            "min_val" : df["RRI"].min(),
            "max_val" : df["RRI"].max()
        }

        meta_data.append(meta_data_row)

        df["is_mutant"] = True if "MD" in element else False
        list_df.append(df)
        
    df_WT = pd.DataFrame(data=raw_data_values_WT)
    df_MT = pd.DataFrame(data=raw_data_values_MT)

    df_metadata = pd.DataFrame(meta_data)
    df_full = pd.concat(list_df, axis=0)

    return df_WT, df_MT, df_metadata, df_full, X_value_MT, X_value_WT

In [22]:
raw_data = "line/"
list_docs_raw = os.listdir(raw_data)

conv_data = "curve/"
list_docs_conv = os.listdir(conv_data)

In [51]:
df_WT_raw, df_MT_raw, df_metadata_raw, df_full_raw, X_value_MT, X_value_WT= create_data(
    list_docs_raw, 
    raw_data)

In [52]:
fig = px.line(df_full_raw, x="Wavenumber", y="RRI-MAS", color='source')
fig.show()

In [59]:
df_WT_conv, df_MT_conv, df_metadata_conv, df_full_conv, X_value_MT_conv, X_value_WT_conv= create_data(
    list_docs_conv, 
    conv_data)

In [60]:
df_full_conv

,Wavenumber,RRI,RRI-MAS,source,is_mutant
0,3998.66622,0.003950,0.000276,spectrum_curve_MD2_28.txt,True
1,3997.33244,0.003970,0.000277,spectrum_curve_MD2_28.txt,True
2,3995.99867,0.003989,0.000278,spectrum_curve_MD2_28.txt,True
3,3994.66489,0.004009,0.000280,spectrum_curve_MD2_28.txt,True
4,3993.33111,0.004030,0.000281,spectrum_curve_MD2_28.txt,True
...,...,...,...,...,...
2994,5.33511,0.004635,0.000214,spectrum_curve_MD3_194.txt,True
2995,4.00133,0.004388,0.000202,spectrum_curve_MD3_194.txt,True
2996,2.66756,0.004170,0.000192,spectrum_curve_MD3_194.txt,True
2997,1.33378,0.003976,0.000183,spectrum_curve_MD3_194.txt,True


In [67]:
scaler = MaxAbsScaler()
data_transform = scaler.fit_transform(df_full_conv[["RRI", "RRI-MAS"]])
data_transform = pd.DataFrame(data=data_transform, columns=["x", "x2"])
df_full_conv["RRI-MAS-ALL"] = data_transform["x"]

In [69]:
fig = px.line(df_full_conv, x="Wavenumber", y="RRI-MAS", color='is_mutant')
fig.show()